<a href="https://colab.research.google.com/github/Elwing-Chou/ml1206/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [ ]:
import glob
dn = os.path.dirname(dataset)
glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")

In [2]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [3]:
train_df = getdata("train")
test_df = getdata("test")

In [ ]:
test_df

In [17]:
# 1. tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [20]:
# tok.index_word
# tok.word_index

In [23]:
# 2. sequenize: 文字化成數字(數字最大值:3000)
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,48,44,2052,911,2,68,5,213,292.0,8.0,3.0,327.0,2186.0,4.0,7.0,7.0,11.0,1906.0,112.0,1074.0,1.0,776.0,1429.0,4.0,1.0,102.0,9.0,35.0,2.0,21.0,61.0,15.0,1.0,33.0,212.0,25.0,74.0,18.0,79.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134,31,1370,140,2373,1,295,2,1.0,780.0,2798.0,906.0,254.0,82.0,2.0,1194.0,5.0,1138.0,1.0,311.0,292.0,8.0,20.0,1.0,372.0,1965.0,1740.0,5.0,2345.0,1627.0,4.0,931.0,2.0,5.0,1445.0,7.0,7.0,156.0,6.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,129,6,161,343,4,477,22,368.0,231.0,14.0,44.0,22.0,25.0,1448.0,24.0,110.0,16.0,87.0,466.0,131.0,1574.0,684.0,2.0,1408.0,364.0,16.0,24.0,220.0,8.0,1.0,127.0,26.0,13.0,35.0,41.0,24.0,1200.0,21.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,63,188,374,34,1174,11,18,33.0,298.0,9.0,13.0,28.0,4.0,65.0,511.0,105.0,9.0,6.0,431.0,3.0,676.0,28.0,37.0,30.0,3.0,1699.0,7.0,7.0,291.0,298.0,12.0,879.0,6.0,71.0,1213.0,2.0,1.0,879.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,771,293,1,17,243,18,188,852.0,5.0,64.0,9.0,9.0,183.0,52.0,218.0,2.0,9.0,13.0,28.0,4.0,1.0,88.0,218.0,204.0,123.0,107.0,1.0,1201.0,9.0,63.0,2226.0,69.0,2.0,90.0,69.0,101.0,1.0,951.0,1478.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,100,1061,5,1,127,10,25,5,132.0,10.0,654.0,10.0,584.0,25.0,1.0,223.0,198.0,30.0,1.0,83.0,270.0,1.0,474.0,387.0,13.0,282.0,426.0,1.0,957.0,1537.0,150.0,9.0,45.0,2259.0,282.0,1.0,1308.0,13.0,150.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,10,101,195,11,6,370,1052,2,523.0,17.0,1613.0,154.0,8.0,362.0,4.0,99.0,26.0,141.0,273.0,1187.0,20.0,24.0,113.0,2.0,195.0,9.0,2149.0,1046.0,1.0,223.0,210.0,4.0,1.0,323.0,41.0,2628.0,41.0,42.0,2.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,84,1053,1325,4,1241,232,254,4,2099.0,530.0,15.0,81.0,4.0,29.0,4.0,740.0,3.0,2730.0,2730.0,15.0,8.0,24.0,2344.0,2051.0,638.0,1.0,1417.0,344.0,72.0,25.0,4.0,769.0,24.0,232.0,5.0,120.0,3.0,508.0,2.0,1149.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,32,318,236,31,32,896,523,2,1467.0,2441.0,4.0,10.0,25.0,107.0,11.0,17.0,378.0,1688.0,208.0,2.0,16.0,254.0,518.0,140.0,10.0,166.0,326.0,2.0,326.0,1739.0,135.0,23.0,35.0,6.0,12.0,123.0,1851.0,23.0,33.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
